# Core ML model export

This notebook converts TensorFlow model to CoreML format. As the second part of this process, there is a separate Jupyter Notebook (can be run only on macOS), that compares results of the conversion with the original TF model.  
More about TensorFlow to CoreML conversion process can be found in a blog post: [TensorFlow to CoreML conversion and model inspection](https://thinkmobile.dev/tensorflow-to-coreml-conversion-and-model-inspection/).

## TensorFlow 2.0 support
At the moment CoreML converter doesn't support TensorFlow 2.0 models. This will change in the future - converter is committed to supporter the latest stable version of TensorFlow. Here is the worth following [issue](https://github.com/tf-coreml/tf-coreml/issues/294) on Github.

In [1]:
!pip install tfcoreml

In [2]:
import tensorflow as tf
import tfcoreml

In [3]:
# TensorFlow 2.0 isn't yet supported. Make sure you use 1.x

print("TensorFlow version {}".format(tf.__version__))
print("Eager mode: ", tf.executing_eagerly())
print("Is GPU available: ", tf.test.is_gpu_available())

TensorFlow version 1.13.1
Eager mode:  False
Is GPU available:  True


In [4]:
#Download MobileNet v2 model from https://www.tensorflow.org/lite/guide/hosted_models

!curl -LO http://download.tensorflow.org/models/tflite_11_05_08/mobilenet_v2_1.0_224.tgz
!curl -LO https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 74.6M  100 74.6M    0     0  52.7M      0  0:00:01  0:00:01 --:--:-- 52.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10484  100 10484    0     0   682k      0 --:--:-- --:--:-- --:--:--  682k


In [5]:
!tar -xvzf mobilenet_v2_1.0_224.tgz

mobilenet_v2_1.0_224.ckpt.data-00000-of-00001
mobilenet_v2_1.0_224.ckpt.index
mobilenet_v2_1.0_224.ckpt.meta
mobilenet_v2_1.0_224_eval.pbtxt
mobilenet_v2_1.0_224_frozen.pb
mobilenet_v2_1.0_224_info.txt
mobilenet_v2_1.0_224.tflite


Learn more about downloaded model. We need to know input and output tensors, so we can point out them in TF CoreML converter. Alternatively you can preview *.pb file in [Netron](https://github.com/lutzroeder/netron).

In [6]:
!cat mobilenet_v2_1.0_224_info.txt

Model: mobilenet_v2_1.0_224
Input: input
Output: MobilenetV2/Predictions/Reshape_1


In [7]:
# Convert TensorFlow model to Core ML format

# Input model definition
IMAGE_INPUT_NAME = ["input:0"]
IMAGE_INPUT_NAME_SHAPE = {'input:0':[1,224,224,3]}
IMAGE_INPUT_SCALE = 1.0/255.0
OUTPUT_NAME = ['MobilenetV2/Predictions/Reshape_1:0']
MODEL_LABELS = 'ImageNetLabels.txt'
TF_FROZEN_MODEL = 'mobilenet_v2_1.0_224_frozen.pb'
# Output model
CORE_ML_MODEL = "mobilenet_v2_1.0_224.mlmodel"

# Convert model and save it as a file
coreml_model = tfcoreml.convert(
        tf_model_path=TF_FROZEN_MODEL,
        mlmodel_path=CORE_ML_MODEL,
        output_feature_names=OUTPUT_NAME,
        image_input_names=IMAGE_INPUT_NAME,
        input_name_shape_dict = IMAGE_INPUT_NAME_SHAPE,
        class_labels=MODEL_LABELS,
        image_scale=IMAGE_INPUT_SCALE
)


Loading the TF graph...
Graph Loaded.
Done.
Now finding ops in the TF graph that can be dropped for inference
Now starting translation to CoreML graph.
Automatic shape interpretation succeeded for input blob input:0
1/316: Analysing op name: MobilenetV2/Conv_1/Conv2D_bn_offset ( type:  Const )
2/316: Analysing op name: Const_71 ( type:  Const )
3/316: Analysing op name: MobilenetV2/expanded_conv_16/project/Conv2D_bn_offset ( type:  Const )
4/316: Analysing op name: Const_100 ( type:  Const )
5/316: Analysing op name: MobilenetV2/expanded_conv_16/expand/Conv2D_bn_offset ( type:  Const )
6/316: Analysing op name: Const_60 ( type:  Const )
7/316: Analysing op name: MobilenetV2/expanded_conv_15/project/Conv2D_bn_offset ( type:  Const )
8/316: Analysing op name: Const_258 ( type:  Const )
9/316: Analysing op name: MobilenetV2/expanded_conv_15/expand/Conv2D_bn_offset ( type:  Const )
10/316: Analysing op name: Const_133 ( type:  Const )
11/316: Analysing op name: MobilenetV2/expanded_conv_1

# Model inspection

Take a look at created files - you should be able to find mobilenet_v2_1.0.224.mlmodel. It's Core ML, ready to be inspected on macOS and implemented in your mobile app.

Currently It is not possible to inspect mlmodel file in Colaboratory (macOS system is required). Continue in Jupyter Notebook "Core ML inspection.ipynb" run on your Mac machine.